In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import logging

# logger = logging.basicConfig(level=logging.INFO)

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

## install dependencies

In [ ]:
# !mamba install -c conda-forge -y make proj-data sentinelsat xmlschema

In [2]:
!pip install elevation sarsen xarray-sentinel

In [3]:
import os

import numpy as np
import rioxarray  # enable the `.rio` accessor
import xarray as xr

from sarsen import apps, scene

## processing definition

In [19]:
# os.makedirs("data", exist_ok=True)
dem_urlpath = "notebooks/output_SRTMGL3.tif"
dem_10m_UTM_urlpath = dem_urlpath.strip(".tif") + "_10m_UTM.tif"

product_folder = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_b716/S1A_IW_GRDH_1SDV_20170508T164830_20170508T164855_016493_01B54C_8604.zarr"
measurement_group = "S01SIWGRD_20170508T164830_0025_A094_8604_01B54C_VH"

## download DEM

In [20]:
import elevation

In [21]:
# ! eio clip -o {dem_urlpath} --bounds 13.1 42. 13.95 42.75

### RESAMPLING UTM DEM

#### 10 meters

In [23]:
dem_raster = scene.open_dem_raster(dem_urlpath)
_ = dem_raster.plot()

RasterioIOError: notebooks/output_SRTMGL3.tif: No such file or directory

In [ ]:
dem_UTM_corners = dict(x=slice(350000, 400000), y=slice(4710000, 4660000))

resolution = (10, 10)

t_srs = dem_raster.rio.estimate_utm_crs()
dem_raster_10m_UTM = dem_raster.rio.reproject(t_srs, resolution=resolution)

dem_raster_10m_UTM = dem_raster_10m_UTM.sel(**dem_UTM_corners)

dem_raster_10m_UTM.rio.to_raster(dem_10m_UTM_urlpath)
dem_raster_10m_UTM

In [15]:
dem_raster.plot()

## download data

In [ ]:
import os

import adlfs
import planetary_computer
import pystac_client
import stackstac


def mirror_folder(fs, bucket, folder, exclude="vh"):
    for path, folders, files in fs.walk(f"{bucket}/{folder}"):
        os.makedirs(path[len(bucket) + 1 :], exist_ok=True)
        for f in files:
            if exclude in f:
                continue
            file_path = os.path.join(path, f)
            lfile_path = file_path[len(bucket) + 1 :]
            if not os.path.isfile(lfile_path):
                print(file_path)
                fs.download(file_path, lfile_path + "~")
                os.rename(lfile_path + "~", lfile_path)

In [ ]:
grd_account_name = "sentinel1euwest"
grd_bucket = "s1-grd"
grd_token = planetary_computer.sas.get_token(grd_account_name, grd_bucket).token

grd_product_folder = f"{grd_bucket}/{product_folder}"

grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(grd_product_folder)

In [ ]:
mirror_folder(grd_fs, grd_bucket, product_folder)

In [ ]:
ds = xr.open_dataset(
    product_folder,
    engine="sentinel-1",
    group=measurement_group,
    # storage_options=storage_options,
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
)
ds

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_bucket = "sentinel1-grd-rtc"
rtc_token = planetary_computer.sas.get_token(rtc_account_name, rtc_bucket).token

rtc_product_folder = f"{rtc_bucket}/{product_folder}"

rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
mirror_folder(rtc_fs, rtc_bucket, product_folder)

## process

#### GTC

In [ ]:
%%time

gtc_path = os.path.basename(product_folder) + ".10m.GTC.tif"

apps.terrain_correction(
    product_folder,
    measurement_group,
    dem_10m_UTM_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
    output_urlpath=gtc_path,
)

!gdaladdo -r average {gtc_path}

gtc_path

In [ ]:
gtc = xr.open_dataarray(gtc_path).drop("band")
gtc.plot(vmax=0.4);

#### RTC

In [ ]:
%%time

rtc_path = os.path.basename(product_folder) + ".10m.RTC.tif"

apps.terrain_correction(
    product_folder,
    measurement_group,
    dem_10m_UTM_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    correct_radiometry="gamma_bilinear",
    output_urlpath=rtc_path,
    grouping_area_factor=(3, 3),
)

!gdaladdo -r average {rtc_path}

rtc_path

In [ ]:
rtc = xr.open_dataarray(rtc_path, cache=False).drop("band")
rtc.plot(vmax=0.4);

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

gtc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")

plt.tight_layout()

In [ ]:
rtc_pc = xr.open_dataarray(
    product_folder + "/measurement/iw-vv.rtc.tiff", cache=False
).drop("band")
rtc_pc = rtc_pc.sel(dem_UTM_corners)
rtc_pc

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

rtc_pc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")

plt.tight_layout()